In [1]:
from pulp import LpProblem, LpVariable, LpMinimize, LpStatus, lpSum, value
from pulp import PULP_CBC_CMD
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import math
import seaborn as sns
import time
import os
import itertools

In [2]:
# Read files
files = glob('../*/*routing_ver2/*.csv')
# print(files)
dfs, name = [], []
for i, file in enumerate(files):
    tmp_df = os.path.splitext(os.path.basename(files[i]))[0]
    name.append(tmp_df)
    df = pd.read_csv(files[i])
    dfs.append(df)
print(name)

['distances', 'locations', 'orders', 'small_distances', 'small_locations', 'small_orders']


In [3]:
locations_df = dfs[4].rename(columns={'k': 'point'})
locations_df = locations_df[['point', 'depo_flag']]
locations_df

,point,depo_flag
0,p,1
1,s1,0
2,s2,0
3,s3,0
4,s4,0


In [4]:
distances_df = dfs[3].rename(columns={'k1': 'i', 'k2': 'j', 't': 'time'})
distances_df = distances_df[distances_df['time'] != 0]
distances_df

,i,j,time
1,p,s1,2.000000
2,p,s2,2.828427
3,p,s3,4.000000
4,p,s4,4.472136
5,s1,p,2.000000
7,s1,s2,2.000000
8,s1,s3,6.000000
9,s1,s4,6.324555
10,s2,p,2.828427
11,s2,s1,2.000000


In [5]:
orders_df = dfs[5].rename(columns={'r': 'pack', 's': 'point', 'w': 'weight', 'b': 'start_d', 'e': 'end_d'})
orders_df

,pack,point,weight,start_d,end_d
0,r1,s1,1000,1,1
1,r2,s2,1000,1,2
2,r3,s3,1000,2,2
3,r4,s4,1500,2,3
4,r5,s1,1500,2,4
5,r6,s2,1500,3,4
6,r7,s3,1500,3,4
7,r8,s4,3000,4,4


In [6]:
s = orders_df[['start_d', 'end_d']].min().min()
e = orders_df[['start_d', 'end_d']].max().max()
schedule = [i for i in range(s, e+1)]
locations = locations_df['point']


In [7]:
df_x = pd.DataFrame()
for d in schedule:
    tmp_df = distances_df.copy()
    tmp_df['d'] = d
    df_x = pd.concat([df_x, tmp_df], axis=0)

df_x['var'] = [LpVariable(f'x_{r.d:02}_{r.i}_{r.j}', cat='Binary') for r in df_x.itertuples()]

In [8]:
df_u = pd.DataFrame(list(itertools.product(days, locations_df['point'])), columns=['d', 'p'])
df_u['var'] = [LpVariable(f'{r.d:02}_{r.p}', cat='Interger') for r in df_u.itertuples()]

NameError: name 'days' is not defined

In [ ]:
df_y = pd.DataFrame(list(itertools.product(days, orders_df['pack'])), columns=['d', 'r'])
df_y['var'] = [LpVariable(f'{r.d:02}_{r.r}', cat='Interger') for r in df_y.itertuples()]

In [ ]:
df_h = pd.DataFrame(days, columns=['d'])
df_h['var'] = [LpVariable(f'h_{r.d}') for r in df_h.itertuples()]

In [ ]:
display(df_x.head(3), df_x.shape[0])
display(df_u.head(3), df_u.shape[0])
display(df_y.head(3), df_y.shape[0])
display(df_h, df_h.shape[0])


,i,j,time,d,var
1,p,s1,2.000000,1,x_01_p_s1
2,p,s2,2.828427,1,x_01_p_s2
3,p,s3,4.000000,1,x_01_p_s3


80

,d,p,var
0,1,p,01_p
1,1,s1,01_s1
2,1,s2,01_s2


20

,d,r,var
0,1,r1,01_r1
1,1,r2,01_r2
2,1,r3,01_r3


32

,d,var
0,1,h_1
1,2,h_2
2,3,h_3
3,4,h_4


4

In [ ]:
m = LpProblem(name='vrp_small', sense=LpMinimize)

for k, v in 

